<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import pandas as pd
import numpy as np

In [57]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-11-04 23:11:09--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.4’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-11-04 23:11:09 (285 KB/s) - ‘house-votes-84.data.4’ saved [18171/18171]



In [58]:
column_headers = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']
df = pd.read_csv('house-votes-84.data', 
                 header=None, 
                 names=column_headers,
                 na_values="?")

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [0]:
df= df.replace({'y': 1, 'n':0})

In [0]:
rep = df[df['party']=='republican']
dem = df[df['party']=='democrat']

In [0]:
bills = column_headers
bills.remove('party')

In [0]:

def votes(bill):
  print(bill)
  print(ttest_ind(rep[bill],dem[bill],nan_policy='omit'))
  print('Democrat mean: ' + str(dem[bill].mean()))
  print('Republican mean: ' + str(rep[bill].mean()))
  print('______________________________________________________________')


Democrats support this bill more than Republicans. 

Since we have a p value of approximately 2.07x10$^{-77}$, we reject the null hypothesis

In [66]:

for bill in bills: 
  votes(bill);

handicapped-infants
Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
Democrat mean: 0.6046511627906976
Republican mean: 0.18787878787878787
______________________________________________________________
water-project
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
Democrat mean: 0.502092050209205
Republican mean: 0.5067567567567568
______________________________________________________________
budget
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
Democrat mean: 0.8884615384615384
Republican mean: 0.13414634146341464
______________________________________________________________
physician-fee-freeze
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
Democrat mean: 0.05405405405405406
Republican mean: 0.9878787878787879
______________________________________________________________
el-salvador-aid
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
Democrat m

# **Null Hypothesis: There is not a statistically significant difference in support between the democrats and republicans**

**aid-to-contras**

Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)

Democrat mean: 0.8288973384030418

Republican mean: 0.15286624203821655

Based on this information, we can say that Democrats support the aid to contras bill more than Republicans

Since we get a p value of 2.82x10$^{-54}$ (<.01), we reject the null hypothesis



**education**

Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)

Democrat mean: 0.14457831325301204

Republican mean: 0.8709677419354839

Based on this information, we can say that Republicans support the education bill more than Democrats

Since we get a p value of 1.88x10$^{-64}$ (<.01), we reject the null hypothesis

**water-project**

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

Democrat mean: 0.502092050209205

Republican mean: 0.5067567567567568

Since we get a p value of .9291556823993485 (>.1), we fail to reject the null hypothesis that there is not a significant difference in support between Democratic and Republican votes on the water project bill.